# Library Import

In [1]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

from tensorflow.keras.preprocessing import image

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

# VGG16

In [2]:
model_vgg_conv = VGG16( weights='imagenet', include_top=False, input_shape=(98,98,3) )

In [3]:
model_vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 98, 98, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 98, 98, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 98, 98, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 49, 49, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 49, 49, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 49, 49, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 24, 24, 128)       0     

In [4]:
for layer in model_vgg_conv.layers:
    layer.trainable = False

In [22]:
dataGen = image.ImageDataGenerator( rescale=1./255)
trainGen = dataGen.flow_from_directory( 'image4', target_size=(98,98), batch_size=1808) # (98,98)인 이유는 model_conv.summary()의 맨 처음을 봐라
x_train, y_train = trainGen.next()

Found 1808 images belonging to 2 classes.


In [23]:
x_train.shape

(1808, 98, 98, 3)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.3,
                                                    stratify=y_train)

In [27]:
x_train.shape, y_train.shape

((1265, 98, 98, 3), (1265, 2))

In [28]:
x_test.shape, y_test.shape

((543, 98, 98, 3), (543, 2))

In [29]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                      y_train,
                                                      test_size=0.3,
                                                      stratify=y_train)

In [30]:
x_train.shape, y_train.shape

((885, 98, 98, 3), (885, 2))

In [31]:
x_valid.shape, y_valid.shape

((380, 98, 98, 3), (380, 2))

## Modeling

In [32]:
model = Sequential()
model.add( model_vgg_conv) 

model.add( Flatten())
model.add( Dense( units=128, activation='relu'))
model.add(Dense(units = 64, activation='relu'))
model.add(Dense(units = 32, activation='relu'))
model.add( Dense( units=2, activation='softmax'))

model.compile( loss="categorical_crossentropy",
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

model.fit( x_train, y_train, epochs=100, validation_data=(x_valid, y_valid))

Epoch 1/100
28/28 [==============================] - 17s 586ms/step - loss: 0.5252 - acc: 0.7401 - val_loss: 0.4022 - val_acc: 0.8289
Epoch 2/100
28/28 [==============================] - 16s 576ms/step - loss: 0.3607 - acc: 0.8452 - val_loss: 0.3284 - val_acc: 0.8500
Epoch 3/100
28/28 [==============================] - 16s 578ms/step - loss: 0.2896 - acc: 0.8814 - val_loss: 0.3612 - val_acc: 0.8395
Epoch 4/100
28/28 [==============================] - 16s 580ms/step - loss: 0.2404 - acc: 0.9017 - val_loss: 0.2761 - val_acc: 0.8737
Epoch 5/100
28/28 [==============================] - 16s 573ms/step - loss: 0.2134 - acc: 0.9186 - val_loss: 0.2598 - val_acc: 0.8816
Epoch 6/100
28/28 [==============================] - 16s 595ms/step - loss: 0.1735 - acc: 0.9401 - val_loss: 0.2505 - val_acc: 0.8921
Epoch 7/100
28/28 [==============================] - 17s 606ms/step - loss: 0.1520 - acc: 0.9480 - val_loss: 0.2464 - val_acc: 0.8974
Epoch 8/100
28/28 [==============================] - 16s 590ms

Epoch 62/100
28/28 [==============================] - 16s 567ms/step - loss: 0.0069 - acc: 0.9977 - val_loss: 0.4095 - val_acc: 0.9105
Epoch 63/100
28/28 [==============================] - 16s 572ms/step - loss: 0.0057 - acc: 0.9989 - val_loss: 0.4825 - val_acc: 0.8868
Epoch 64/100
28/28 [==============================] - 16s 569ms/step - loss: 0.0033 - acc: 0.9989 - val_loss: 0.3870 - val_acc: 0.9105
Epoch 65/100
28/28 [==============================] - 16s 570ms/step - loss: 0.0072 - acc: 0.9977 - val_loss: 0.3977 - val_acc: 0.9053
Epoch 66/100
28/28 [==============================] - 16s 568ms/step - loss: 0.0090 - acc: 0.9977 - val_loss: 0.4075 - val_acc: 0.9105
Epoch 67/100
28/28 [==============================] - 16s 580ms/step - loss: 0.0062 - acc: 0.9989 - val_loss: 0.4218 - val_acc: 0.9105
Epoch 68/100
28/28 [==============================] - 16s 571ms/step - loss: 0.0053 - acc: 0.9989 - val_loss: 0.6572 - val_acc: 0.8763
Epoch 69/100
28/28 [==============================] - 1

## Prediction

In [34]:
x_test.shape, x_test

((543, 98, 98, 3),
 array([[[[0.75294125, 0.68235296, 0.5882353 ],
          [0.76470596, 0.6784314 , 0.5882353 ],
          [0.7843138 , 0.7019608 , 0.5882353 ],
          ...,
          [0.7019608 , 0.6784314 , 0.5529412 ],
          [0.69803923, 0.68235296, 0.54509807],
          [0.7137255 , 0.7411765 , 0.5686275 ]],
 
         [[0.64705884, 0.5764706 , 0.48235297],
          [0.7176471 , 0.63529414, 0.52156866],
          [0.8078432 , 0.72156864, 0.6392157 ],
          ...,
          [0.6784314 , 0.654902  , 0.5294118 ],
          [0.7058824 , 0.70980394, 0.5882353 ],
          [0.70980394, 0.7490196 , 0.50980395]],
 
         [[0.77647066, 0.7058824 , 0.6117647 ],
          [0.78823537, 0.72156864, 0.61960787],
          [0.73333335, 0.6666667 , 0.5647059 ],
          ...,
          [0.69411767, 0.68235296, 0.52156866],
          [0.7176471 , 0.7490196 , 0.59607846],
          [0.6784314 , 0.7372549 , 0.4666667 ]],
 
         ...,
 
         [[0.70980394, 0.7411765 , 0.7490196 ],

### make a prediction

In [36]:
model.predict( x_test).argmax(axis=1)

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,

### Label(정답) 확인

In [38]:
trainGen.class_indices

{'lion': 0, 'tiger1': 1}

## Accuracy

### Train data

In [39]:
model.evaluate(x_train, y_train)

28/28 [==============================] - 12s 414ms/step - loss: 0.0016 - acc: 0.9989


[0.0016263133147731423, 0.9988700747489929]

### Test data

In [40]:
model.evaluate(x_test, y_test)

17/17 [==============================] - 7s 399ms/step - loss: 0.5485 - acc: 0.9134


[0.5484875440597534, 0.9134438037872314]

## 전혀 다른 새로운 데이터 Test

```
새로운 데이터를 다시 크롤링 ==> 새로운 Test data ==> accuracy 계산
```

In [41]:
dataGen = image.ImageDataGenerator( rescale=1./255)
testGen = dataGen.flow_from_directory( 'image5', target_size=(98,98), batch_size=270)
x_test_new, y_test_new = testGen.next()

Found 270 images belonging to 2 classes.


## Accuracy

In [42]:
model.evaluate(x_test_new, y_test_new)

9/9 [==============================] - 3s 371ms/step - loss: 1.0392 - acc: 0.8667


[1.0392488241195679, 0.8666666746139526]